<a href="https://colab.research.google.com/github/Ashish-Garg06/Classic-tic-tac-toe/blob/main/facial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless pillow --quiet

import numpy as np
import cv2
from IPython.display import display, clear_output
from google.colab.output import eval_js
from PIL import Image
from base64 import b64decode
import io
import urllib.request



def download_xml(url, path):
    urllib.request.urlretrieve(url, path)

download_xml(
    "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml",
    "face.xml"
)
download_xml(
    "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_smile.xml",
    "smile.xml"
)
download_xml(
    "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml",
    "eye.xml"
)

face_detector = cv2.CascadeClassifier("face.xml")
smile_detector = cv2.CascadeClassifier("smile.xml")
eye_detector = cv2.CascadeClassifier("eye.xml")

def estimate_emotion(gray, x, y, w, h):
    roi = gray[y:y+h, x:x+w]

    smiles = smile_detector.detectMultiScale(roi, 1.5, 18)
    eyes   = eye_detector.detectMultiScale(roi, 1.1, 3)

    if len(smiles) > 0:
        return "Happy"

    if len(eyes) == 0:
        return "Sleepy"

    if w / h > 1.3:
        return "Surprised"

    return "Neutral"




Stopped.


In [ ]:

def capture_image():
    js_code = """
    async function captureFrame() {
      const video = document.createElement('video');
      const stream = await navigator.mediaDevices.getUserMedia({video:true});
      video.srcObject = stream;
      await video.play();

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      const ctx = canvas.getContext('2d');
      ctx.drawImage(video, 0, 0);

      stream.getTracks().forEach(t => t.stop());
      return canvas.toDataURL('image/jpeg', 0.5); // Changed quality to 0.5
    }
    captureFrame();
    """
    data = eval_js(js_code)            # <---- FIXED!
    img_bytes = b64decode(data.split(",")[1])
    img = Image.open(io.BytesIO(img_bytes))
    return np.array(img)

In [ ]:

print("Live Emotion Detection Ready. Allow webcam access.")

try:
    while True:
        frame = capture_image()
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)

        faces = face_detector.detectMultiScale(gray, 1.3, 4)

        for (x, y, w, h) in faces:
            emotion = estimate_emotion(gray, x, y, w, h)

            cv2.rectangle(frame_bgr, (x,y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame_bgr, emotion, (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.8, (0,255,0), 2)

        img_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        display(Image.fromarray(img_rgb))
        clear_output(wait=True)

except KeyboardInterrupt:
    print("Stopped.")


Stopped.
